# Simple Policy Gradient

This notebook contains the implementation of the Simple Policy Gradient Algorithm using TensorFlow.
<br/>
This notebook is created while going through the official Spinning up in Deep RL Docs.

In [1]:
# Required modules
!pip install gym

    100% |████████████████████████████████| 1.5MB 2.3MB/s 
    100% |████████████████████████████████| 1.0MB 18.9MB/s 
  Running setup.py bdist_wheel for gym ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/6c/3a/0e/b86dee98876bb56cdb482cc1f72201035e46d1baf69d10d028
Successfully built gym


In [0]:
# Import required modules
import numpy as np
import tensorflow as tf
import gym
from gym import 

In [0]:
# Arguments
env_name = 'CartPole-v0'
render = True
lr = 1e-2

In [4]:
# Create the env
env = gym.make('CartPole-v0')

/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [0]:
# Get the action space size and observation space size
